In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv',parse_dates='datetime',index_col='datetime')
test = pd.read_csv('data/test.csv',parse_dates='datetime',index_col='datetime')

In [3]:
train[:5]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0,0,1,1


In [4]:
test[:5]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
datetime,,,,,,,,
2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [5]:
train['hour'] = train.index.hour
train['month'] = train.index.month
train['year'] = train.index.year - 2011
train['day_of_week'] = train.index.weekday
train[:3]
test['hour'] = test.index.hour
test['month'] = test.index.month
test['year'] = test.index.year - 2011
test['day_of_week'] = test.index.weekday

In [6]:
from sklearn.linear_model import Lasso

In [7]:
clf = Lasso()

In [8]:
clf.fit(train['atemp'],train['count']);

In [9]:
prediction = clf.predict(train['count'])

In [10]:
print prediction

191.574131913


In [11]:
import numpy as np
def error(predictions,actual):
    print predictions[:10]
    print np.array(actual)[:10]
    err = 0
    for i, pred in enumerate(predictions):
        err += (np.log1p(pred)-np.log1p(actual[i]))**2
    J = (err/len(predictions))**.5
    return J

err=0
for i in train['count']:
    err += (np.log1p(prediction)-np.log1p(i))**2

print (err/1)**.5
#print error(prediction, train['count'])

163.723867294


In [12]:
import numpy as np
humidity = Lasso()
humidity.fit(train['humidity'],train['count'])
hPredict = humidity.predict(train['count'])
err=0
for i in train['count']:
    err += (np.log1p(hPredict)-np.log1p(i))**2

print (err/1)**.5

163.723867294


In [13]:
from sklearn import cross_validation
kf = cross_validation.KFold(len(train),5,shuffle=True)
print kf

sklearn.cross_validation.KFold(n=10886, n_folds=5, shuffle=True, random_state=None)


In [14]:
import numpy as np
twoFeats = Lasso()
features = ['humidity', 'atemp']
twoFeats.fit(train[features],train['count'])
predict = twoFeats.predict(train['count'])

print error(predict)

ValueError: shapes (10886,) and (2,) not aligned: 10886 (dim 0) != 2 (dim 0)

In [15]:
twoFeats

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute='auto', tol=0.0001,
   warm_start=False)

In [17]:
predict = twoFeats.predict(train[features])

In [18]:
predict

array([  63.04235387,   59.76404353,   59.76404353, ...,  131.82921956,
        144.00659944,  123.74544336])

In [19]:
train['count']

datetime
2011-01-01 00:00:00     16
2011-01-01 01:00:00     40
2011-01-01 02:00:00     32
2011-01-01 03:00:00     13
2011-01-01 04:00:00      1
2011-01-01 05:00:00      1
2011-01-01 06:00:00      2
2011-01-01 07:00:00      3
2011-01-01 08:00:00      8
2011-01-01 09:00:00     14
2011-01-01 10:00:00     36
2011-01-01 11:00:00     56
2011-01-01 12:00:00     84
2011-01-01 13:00:00     94
2011-01-01 14:00:00    106
2011-01-01 15:00:00    110
2011-01-01 16:00:00     93
2011-01-01 17:00:00     67
2011-01-01 18:00:00     35
2011-01-01 19:00:00     37
2011-01-01 20:00:00     36
2011-01-01 21:00:00     34
2011-01-01 22:00:00     28
2011-01-01 23:00:00     39
2011-01-02 00:00:00     17
2011-01-02 01:00:00     17
2011-01-02 02:00:00      9
2011-01-02 03:00:00      6
2011-01-02 04:00:00      3
2011-01-02 06:00:00      2
                      ... 
2012-12-18 18:00:00    525
2012-12-18 19:00:00    353
2012-12-18 20:00:00    268
2012-12-18 21:00:00    168
2012-12-18 22:00:00    132
2012-12-18 23:00:00

In [20]:
error(predict, train['count'])

[  63.04235387   59.76404353   59.76404353   80.02519961   80.02519961
   67.84781973   59.76404353   36.71260254   80.02519961  101.54948508]
[16 40 32 13  1  1  2  3  8 14]


nan

In [21]:
predict[predict < 0] = 0

In [22]:
error(predict, train['count'])

[  63.04235387   59.76404353   59.76404353   80.02519961   80.02519961
   67.84781973   59.76404353   36.71260254   80.02519961  101.54948508]
[16 40 32 13  1  1  2  3  8 14]


1.3921726746499172